In [1]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [9]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: - ^C
failed

CondaError: KeyboardInterrupt

Solving environment: / ^C
failed

CondaError: KeyboardInterrupt

Folium installed
Libraries imported.


In [4]:
address = 'Hyderabad, India'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Hyderabad home are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


The geograpical coordinate of Hyderabad home are 17.38878595, 78.4610647345315.


In [11]:
CLIENT_ID = 'ZZXVI3CZNZ1G4PY1WUFCZ1HXTRKQPVTE20CZOCRXNA3JI0S3' # your Foursquare ID
CLIENT_SECRET = 'JD4JUKMRCQCYPMLB5NMPTTJ0ATRGOEIKYT3KM4TDDQSJSNEU' # your Foursquare Secret
VERSION = '20190902' # Foursquare API version

In [12]:
neighborhood_latitude=17.38878595
neighborhood_longitude=78.4610647345315

In [13]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ZZXVI3CZNZ1G4PY1WUFCZ1HXTRKQPVTE20CZOCRXNA3JI0S3&client_secret=JD4JUKMRCQCYPMLB5NMPTTJ0ATRGOEIKYT3KM4TDDQSJSNEU&v=20190902&ll=17.38878595,78.4610647345315&radius=500&limit=100'

In [15]:


# results display is hidden for report simplification 
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d6dd2e56adbf5002cdd6809'},
 'response': {'headerLocation': 'Hyderabad',
  'headerFullLocation': 'Hyderabad',
  'headerLocationGranularity': 'city',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 17.393285954500005,
    'lng': 78.46577143885987},
   'sw': {'lat': 17.384285945499993, 'lng': 78.45635803020313}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c6c20ef23c1a1cd47b218cf',
       'name': 'Al-humdulillah Hotel',
       'location': {'lat': 17.390728238550555,
        'lng': 78.46299202576266,
        'labeledLatLngs': [{'label': 'display',
          'lat': 17.390728238550555,
          'lng': 78.46299202576266}],
        'distance': 297,
        'cc': 'IN',
        'country': 'India',
        'formattedAddress': ['I

In [16]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
venues = results['response']['groups'][0]['items']
SGnearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]
# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)
# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.shape

(4, 4)

In [18]:
SGnearby_venues.head(10)

,name,categories,lat,lng
0,Al-humdulillah Hotel,Indian Restaurant,17.390728,78.462992
1,Milan Juice Center,Snack Place,17.392266,78.458102
2,Feel in Saudi,Afghan Restaurant,17.390702,78.463603
3,Chinese Kitchen,Chinese Restaurant,17.391587,78.457540


In [20]:
latitude=17.390728
longitude=78.462992
# create map of Singapore place  using latitude and longitude values
map_sg = folium.Map(location=[latitude, longitude], zoom_start=18)
# add markers to map
for lat, lng, label in zip(SGnearby_venues['lat'], SGnearby_venues['lng'], SGnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=30,
        radius=7,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.6,
    ).add_to(map_sg)  
    
map_sg

